In [1]:
# Datasets grabbed from https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
# Let's look at the problem of forecasting stocks

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from data import picture, dataShift, get_x_y, get_mdl_inputs, calc_y, get_trn_val

In [3]:
# Let's import one of the datasets from the link.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('aadr.us.csv')

numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']


In [4]:
draw = False
if draw == True:
    picture(df)

In [5]:
cleaner = dataShift(df, numeric_cols)

In [6]:
x, y, past_cols = get_x_y(numeric_cols, cleaner)

In [7]:
y_diff, x_with_current = get_mdl_inputs(cleaner, numeric_cols, past_cols)

In [8]:

x_with_current.shape, y_diff.shape

((1551, 70), (1551,))

In [9]:
# mdl = Ridge().fit(x_with_current, y_diff)
# yhat = mdl.predict(x_with_current)
# mdl = Ridge().fit(x, y)
# yhat = mdl.predict(x)


# predictive_correlation = np.corrcoef(yhat, y_diff)
# predictive_correlation[1,0]

# treemdl = GradientBoostingRegressor().fit(x_with_current, y_diff)
# treeyhat = treemdl.predict(x_with_current)

# predictive_correlation_tree = np.corrcoef(treeyhat, y_diff)
# predictive_correlation_tree[1,0]


# nnmdl = MLPRegressor([100]).fit(x_with_current, y_diff)
# nnyhat = nnmdl.predict(x_with_current)

# predictive_correlation_nn = np.corrcoef(nnyhat, y_diff)
# predictive_correlation_nn[1,0]

In [10]:
x_trn, y_trn, x_val, y_val = get_trn_val(x_with_current, numeric_cols, 0.6)

In [15]:
mdl = GradientBoostingRegressor().fit(x_trn, y_trn)
yhat = mdl.predict(x_val)
predictive_correlation = np.corrcoef(yhat, y_val)
predictive_correlation[1,0]


0.02199304343509962

In [12]:
mdl = Ridge().fit(x_trn, y_trn)
yhat = mdl.predict(x_trn)
predictive_correlation = np.corrcoef(yhat, y_trn)
predictive_correlation[1,0]

0.1602721246843962

In [28]:
def validation(mdl,x_val,y_val):
    money = 100
    predictions = mdl.predict(x_val)
    assignments = predictions > predictions.mean()
    for assignment,y in zip(assignments, y_val):
        money = money * (1-assignment) + assignment * money * (1 + y)
    return money

In [29]:
validation(mdl,x_val,y_val)

159.29273834017664

In [31]:
def baseline(y_val):
    money = 100
    for y in y_val:
        money = money * (1+y) 
    return money

In [33]:
baseline(y_val)

180.92187173160758

In [43]:
for pct_train in [.2,.3,.4,.5,.6,.7,.8,.9]:
    x_trn, y_trn = get_trn_val(x, numeric_cols, pct_train)

KeyError: "None of [Index(['High', 'Low'], dtype='object')] are in the [columns]"

In [36]:
x_trn, y_trn = get_trn_val(.8)

NameError: name 'get_split' is not defined